## Notebook for ML training 

In [3]:
import utils 
import json
import gzip
from servicex import deliver
from servicex import query as q
from servicex_analysis_utils import ds_type_resolver

In [6]:
fname = "file_metadata.json.gz"
with gzip.open(fname) as f:
    dataset_info = json.loads(f.read().decode())

Custom utilities to list sample categories with TopCP outputs

In [7]:
samples_with_ntuples=utils.check_for_ntuples(dataset_info)

In [8]:
def build_ntuple_list(metadata: dict, categories: list[str] | str):
    if isinstance(categories, str):
        categories = [categories]
    containers={}
    
    for category in categories:
        dataset=metadata.get(category, {})
    
        for container, info in dataset.items():
            if info["output"] is None:
                continue
            did, _, campaign = utils.dsid_rtag_campaign(container)
            containers[f"{category}_{did}_{campaign}"] = "user.alheld:"+info["output"][:-1]

    return containers

In [9]:
a=list(samples_with_ntuples.keys())

In [10]:
ntuples = build_ntuple_list(dataset_info, a)

In [11]:
len(ntuples)

364

In [13]:
print(list(ntuples.items())[0])

('Wt_601352_mc20d', 'user.alheld:user.alheld.601352.PhPy8EG.DAOD_PHYSLITE.e8547_s4231_r13144_p6697.IC-v1_output')


In [14]:
simple_query = q.FuncADL_Uproot().FromTree("reco").Select(lambda e: {"jet_eta": e["jet_eta"]})

In [15]:
def build_spec(dataset, request_name, query):
        queries = []

        for sample, name in zip(dataset, request_name):

            queries.append(
                {
                    "Name": name,
                    "Dataset": ds_type_resolver(sample),
                    "Query": query,
                }
            )

        spec = {"General": {"Delivery": "LocalCache"}, "Sample": queries}
        return spec

In [16]:
spec=build_spec(ntuples.values(), ntuples.keys(), simple_query)

In [17]:
len(spec["Sample"])

364

In [ ]:
deliver(spec)

Output()